In [ ]:
import xarray as xr
import requests
import netCDF4
import boto3
import boto3
from botocore import UNSIGNED
from botocore.config import Config
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature


In [ ]:
bucket_name = 'noaa-goes16'
product_name = 'ABI-L1b-RadF'
year = 2019
day_of_year = 79
hour = 14
band = 3

In [ ]:
# Initialize s3 client. 
s3_client = boto3.client('s3', config=Config(signature_version=UNSIGNED))

In [ ]:
def get_s3_keys(bucket, s3_client, prefix = ''):
    """
    Generate the keys in an S3 bucket.

    :param bucket: Name of the S3 bucket.
    :param prefix: Only fetch keys that start with this prefix (optional).
    """
    
    kwargs = {'Bucket': bucket}

    if isinstance(prefix, str):
        kwargs['Prefix'] = prefix

    while True:
        resp = s3_client.list_objects_v2(**kwargs)
        for obj in resp['Contents']:
            key = obj['Key']
            if key.startswith(prefix):
                yield key

        try:
            kwargs['ContinuationToken'] = resp['NextContinuationToken']
        except KeyError:
            break

In [ ]:
keys = get_s3_keys(bucket_name,
                   s3_client,
                   prefix = f'{product_name}/{year}/{day_of_year:03.0f}/{hour:02.0f}/OR_{product_name}-M3C{band:02.0f}'
                  )


key = [key for key in keys][0] # selecting the first measurement taken within the hour

In [ ]:
# resp = requests.get(f'https://{bucket_name}.s3.amazonaws.com/{key}')


In [ ]:
file_name = key.split('/')[-1].split('.')[0]
nc4_ds = netCDF4.Dataset(file_name, memory = resp.content)
store = xr.backends.NetCDF4DataStore(nc4_ds)
DS = xr.open_dataset(store)

In [ ]:
print(DS.attrs)

In [ ]:
# fill_value = -9999
# DS['Rad'] = DS['Rad'].fillna(fill_value)

# fig = plt.figure(figsize=(15, 12))

# # Generate a Cartopy projection
# lc = ccrs.LambertConformal(central_longitude=-97.5, standard_parallels=(38.5, 38.5))

# ax = fig.add_subplot(1, 1, 1, projection=lc)
# ax.set_extent([-135, -60, 10, 65], crs=ccrs.PlateCarree())

# x = DS.x.values
# y = DS.y.values
# goes_imager_projection = DS['Rad'].coords['x'].attrs['projection']

# vmin, vmax = 0, 1022  # Adjust these values based on your data range
# ax.pcolormesh(x, y, DS.Rad, transform=goes_imager_projection, cmap='gray', vmin=vmin, vmax=vmax)

# ax.coastlines(resolution='50m', color='black', linewidth=0.5)
# ax.add_feature(cfeature.STATES, linewidth=0.5)

# plt.title('GOES-16 Radiance', loc='left', fontweight='bold', fontsize=15)
# plt.title('{}'.format(DS.time_coverage_start), loc='right')

# plt.show()

In [ ]:
# print(DS.Rad.shape)
# print(DS.Rad.isel(x=slice(0, 10), y=slice(0, 10)))

In [ ]:
fig = plt.figure(figsize=(12, 12))
plt.imshow(DS.Rad, cmap='gray')
plt.axis('off')
plt.savefig(f'{file_name}.png', dpi=300, facecolor='w', edgecolor='w')